In [ ]:
!pip install shexer

sheXer can generate different types of outputs:
* SheX (in compact syntax)
* SHACL (in turtle)
* Statistics (as annotations in ShEx)
* Examples of conformance (as annotations in ShEx).
* UML visualizations.
* RDF-Config files
* Shapes for federation.

In this notebook, we provide some examples on how to produce them all.

In [ ]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE_ITER, SHACL_TURTLE, SHEXC, SHAPE_EXAMPLES, CONSTRAINT_EXAMPLES, ALL_EXAMPLES, RATIO_INSTANCES, ABSOLUTE_INSTANCES, MIXED_INSTANCES
import requests

def remote_to_local(url, local_path):
  response = requests.get(url)
  if response.status_code == 200:
      with open(local_path, "w", encoding="utf-8") as out_stream:
          out_stream.write(response.text)

INPUT_GRPAH_PATH = "local_file.ttl"
def default_namespaces():
    return {"http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf"
            }

remote_to_local("https://raw.githubusercontent.com/weso/shexer/refs/heads/master/test/t_files/t_graph_1.ttl",
                INPUT_GRPAH_PATH)

print("# We will work with this graph:\n")

with open(INPUT_GRPAH_PATH) as out_stream:
  print(out_stream.read())

In [ ]:
# Generation of SHACL to file

shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER)
shaper.shex_graph(output_file="shacl_shapes.ttl",  # Provide a disk path to save the results with this parameter
                  output_format=SHACL_TURTLE) # Set this param with this value to generate SHACL shapes

with open("shacl_shapes.ttl") as in_stream:
  print(in_stream.read())  # Just checking the content of the file generated



In [ ]:
# Same thing, but without saving to file. Instead, result returned as string

shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER)
result = shaper.shex_graph(string_output=True,  # If you set this to True, the method will return string
                           output_format=SHACL_TURTLE) # you can indicate a fiel anyway. If you do, the results will be saved to disk regardless of wheter they were returned in a string

print(result)
print("---------------------------")
with open("shacl_shapes.ttl") as in_stream:
  print(in_stream.read())  # Just checking the content of the file generated

In [ ]:
# Same input graph, but now we generate results in ShEx
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER)
result = shaper.shex_graph(string_output=True,
                           output_format=SHEXC) # SHEXC is the default value. If you dont set this parameter, results will be generated in ShEx.
print(result)


In [ ]:
# Note that ShEx outputs are commented with statistical information.
# We can disable such information and just get shapes


shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    disable_comments=True,  # Use this to avoid getting comments on shapes.
    input_format=TURTLE_ITER)
result = shaper.shex_graph(string_output=True)
print(result)


In [ ]:
# We could also make comments even richer. shapes can be annotated with examples
# at constraint level (examples of nodes matching node constraints)
# or at shape level (examples nodes matching shapes).

# You can get only shapes
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    disable_comments=False,  # Default value, you could just omit this when it is False
    input_format=TURTLE_ITER,
    examples_mode=SHAPE_EXAMPLES) # Ste this parameter to some of the values allowed to get shapes annotated with examples
result = shaper.shex_graph(string_output=True)
print("_______examples at shape level________")
print(result)

# ... only constraints
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    disable_comments=False,  # Default value, you could just omit this when it is False
    input_format=TURTLE_ITER,
    examples_mode=CONSTRAINT_EXAMPLES) # Ste this parameter to some of the values allowed to get shapes annotated with examples
result = shaper.shex_graph(string_output=True)
print("_______examples at constraint level________")
print(result)

# ... or both things at a time.
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    disable_comments=False,  # Default value, you could just omit this when it is False
    input_format=TURTLE_ITER,
    examples_mode=ALL_EXAMPLES) # Ste this parameter to some of the values allowed to get shapes annotated with examples
result = shaper.shex_graph(string_output=True)
print("_______examples at ALL level________")
print(result)

In [ ]:
# You can also tune if you want absolute, relative or both types for the generated stats

# Relative
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER,
    instances_report_mode=RATIO_INSTANCES) # Default value, you could omit this in case you prefer ratio
result = shaper.shex_graph(string_output=True)
print("_______Relative stats________")
print(result)

# Absolute
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER,
    instances_report_mode=ABSOLUTE_INSTANCES) # For absolute values
result = shaper.shex_graph(string_output=True)
print("_______Absolute stats________")
print(result)

# Both
shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER,
    instances_report_mode=MIXED_INSTANCES) # For both at a time
result = shaper.shex_graph(string_output=True)
print("_______Both stat types________")
print(result)

In [ ]:
# Lets generate now UML visualizations instead of shapes

from PIL import Image
from IPython.display import display

UML_PATH = 'uml_shapes.jpg'


shaper = Shaper(
    graph_file_input=INPUT_GRPAH_PATH,
    namespaces_dict=default_namespaces(),
    all_classes_mode=True,
    input_format=TURTLE_ITER)
shaper.shex_graph(to_uml_path=UML_PATH)  # Just set a value for this parameter and the visualization will be generated

image = Image.open(UML_PATH)
display(image)

In [ ]:
# Let's generate now RDF-Config files. Read about RDF-Config at: https://github.com/dbcls/rdf-config
# 3 RDF-Config files are generated. model, prefixes and endpoint. In case we
# use RDF input instead of SPARQL endpoints (as in this case), endpont.yaml won't be generated
# You must tell sheXer the directory in which you want the files to be writen
# but you can also obtain the output via string.

TARGET_DIR = "."

shaper = Shaper(graph_file_input=INPUT_GRPAH_PATH,
                all_classes_mode=True,
                input_format=TURTLE_ITER,
                namespaces_dict=default_namespaces(),
                examples_mode=ALL_EXAMPLES,  # When generating RDF-COnfig files, make sure to add this, as examples are a core part of the RDF-Config YAML files
                                             # and sheXer won't track them unless it is told to do so.
                instances_report_mode=MIXED_INSTANCES)


result = shaper.shex_graph(rdfconfig_directory=TARGET_DIR, # Set this property to some value to generate rdf-config files
                            string_output=True)

print(result)



# Shapes for federated queries

sheXer can also generate some shapes which are meant to be used to document
structures in scenarios of potential federated queries.

Such feature requires to instantiate an object with several parameters and
it is explained in a standalone example. Check it at [this link](https://github.com/weso/shexer/blob/master/doc/Example_federated_shapes.ipynb)